# Introduction to `networkx`

**NetworkX: Network Analysis in Python**

<img src="https://networkx.org/_static/networkx_logo.svg" width="400px" />

> NetworkX is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

Mainly, `networkx` provides data structures for undirected and directed graphs
as well as multigraphs (graphs with multiple edges between nodes) for which many
standard graph algorithms are implemented (such as shortest paths, clustering,
cycle detection, and more).

:::{note}
Documentation for this package is available at https://networkx.github.io/.
:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install numpy networkx pandas matplotlib
```
:::

In [ ]:
import warnings

warnings.filterwarnings("ignore")

Let's perform some network analysis on this simple graph:

<img src="https://raw.githubusercontent.com/fneum/data-science-for-esm/main/data-science-for-esm/network.png" width="300px" />

## Simple Graphs

In [ ]:
import numpy as np

Say we want to calculate the so-called *Laplacian* $L$ of this graph based on its incidence
matrix $K$, which is an $N\times N$ matrix defined as $L=KK^\top$ for an
undirected graph. The Laplacian matrix of a graph is a representation that
captures the connectivity and structure of the graph by quantifying the
difference between the degree of each vertex and the adjacency relationships
with its neighbouring vertices. We first need to write down the incidence matrix
$K$ as a `np.array`. Let's also use the convention that edges are oriented such
that they are directed at the node with the higher label value (i.e. from node 1
to node 2, not vice versa).

In [ ]:
K = np.array(
    [
        [1, -1, 0, 0],
        [1, 0, -1, 0],
        [0, 1, -1, 0],
        [0, 0, 1, -1],
    ]
).T
K

In [ ]:
L = K.dot(K.T)
L

This is all fine for small graphs, but inconvient for larger graphs. Let's take the help from `networkx`!

## The `Graph` object

First, let's import the library. It is commonly imported under the alias `nx`.

In [ ]:
import networkx as nx

This is how we can create an empty graph with no nodes and no edges.

In [ ]:
G = nx.Graph()

### Nodes

We can add one node at a time,

In [ ]:
G.add_node(1)

with attributes

In [ ]:
G.add_node(2, country="DE")

or add nodes from a list

In [ ]:
G.add_nodes_from([3, 4])

We can also add nodes along with node attributes if your container yields 2-tuples of the form `(node, node_attribute_dict)`:

In [ ]:
G.add_nodes_from(
    [
        (5, {"color": "red"}),
        (6, {"color": "green"}),
    ]
)

### Edges

`G` can also be grown by adding one edge at a time,

In [ ]:
G.add_edge(1, 2)

even with attributes

In [ ]:
G.add_edge(3, 4, weight=2)

or by adding a list of edges,

In [ ]:
G.add_edges_from([(1, 3), (2, 5)])

or as a 3-tuple with 2 nodes followed by an edge attribute dictionary

In [ ]:
G.add_edges_from([(2, 3, {"weight": 3})])

### Examining elements of a graph

We can examine the nodes and edges.

In [ ]:
G.nodes

In [ ]:
G.number_of_nodes()

In [ ]:
G.edges

In [ ]:
G.number_of_edges()

### Accessing graph elements

Access an edge:

In [ ]:
G.edges[2, 3]

Access an attribute of an edge:

In [ ]:
G.edges[2, 3]["weight"]

Find all neighbours of node 1:

In [ ]:
G[1]

### Removing elements

One can remove nodes and edges from the graph in a similar fashion to adding. Use methods `G.remove_node()`, `G.remove_nodes_from()`, `G.remove_edge()` and `G.remove_edges_from()`, e.g.

In [ ]:
G.remove_node(5)

In [ ]:
try:
    G.remove_edge(2, 5)
except Exception as e:
    print(e)

:::{note}
Removing a node will also remove all adjacent edges!
:::

You can remove all nodes and edges with

In [ ]:
# G.clear()

### Visualising graphs

A basic drawing function for networks is also available in `networkx`

In [ ]:
nx.draw(G)

with options for labeling graph elements

In [ ]:
nx.draw(G, with_labels=True)

and integration to `matplotlib`

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(5, 5))
nx.draw(G, with_labels=True, ax=ax, node_color="green", font_weight="bold")

:::{note}
For a full list of arguments of the function see
https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html
:::

### Analysing graphs

The `networkx` library comes with many functions to analyse graphs. Here are a few examples we're going to need for linearised power flow calculations in electricity transmission networks:

Are all nodes in the network connected with each other?

In [ ]:
nx.is_connected(G)

What are the components that are connected / isolated?

In [ ]:
list(nx.connected_components(G))

Is the network planar? I.e. can the graph be drawn such that edges don't cross?

In [ ]:
nx.is_planar(G)

What is the frequency of degrees in the network?

In [ ]:
nx.degree_histogram(G)

In [ ]:
import pandas as pd

pd.Series(nx.degree_histogram(G))

What is the *adjacency matrix*? (Careful, `networkx` will yield a weighted adjacency matrix by default!)

In [ ]:
A = nx.adjacency_matrix(G, weight=None).todense()
A

What is the *incidence matrix*? (Careful, `networkx` will yield a incidence matrix without orientation by default!)

In [ ]:
nx.incidence_matrix(G, oriented=True).todense()

What is the *Laplacian matrix*? (Careful, `networkx` will yield a weighted adjacency matrix by default!)

In [ ]:
L = nx.laplacian_matrix(G, weight=None).todense()
L

Find a cycle basis (i.e. a collection of independent cycles through which all other cycles can be represented through linear combination):

In [ ]:
nx.cycle_basis(G)

This function returns a list of sequences. Each sequence indicates a series of nodes to traverse for the respective cycle.

## Example: European Transmission Network

Let's now apply what we have learned to a real-world example. 
In this example, we are going to load a slightly simplified dataset of the European high-voltage transmission network. In this dataset, HVDC links and back-to-back converters have been left out, and the data only shows AC transmission lines.

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/898dEKqG6XEDqqS/download/nodes.csv"
nodes = pd.read_csv(url, index_col=0)
nodes.head(5)

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/FmFrJkiWpg2QcQA/download/edges.csv"
edges = pd.read_csv(url, index_col=0)
edges.head(5)

`networkx` provides a utility function `nx.from_pandas_edgelist()` to build a network from edges listed in a `pandas.DataFrame`:

In [ ]:
N = nx.from_pandas_edgelist(edges, "bus0", "bus1", edge_attr=["x_pu", "s_nom"])

We can get some basic info about the graph:

In [ ]:
print(N)

In [ ]:
pd.Series(nx.degree_histogram(N)).plot.bar()
plt.xlabel("Degree (number of connections)")
plt.ylabel("Number of nodes")

In [ ]:
degrees = [val for node, val in N.degree()]
np.mean(degrees)

The `nodes` DataFrame provides us with the coordinates of the graph's nodes. To make the `nx.draw()` function use these coordinates, we need to bring them into a particular format.

```
{nodeA: (x, y),
 nodeB: (x, y),
 nodeC: (x, y)}
```

In [ ]:
pos = nodes.apply(tuple, axis=1).to_dict()

Let's just look at the first 5 elements of the dictionary to check:

In [ ]:
{k: pos[k] for k in list(pos.keys())[:5]}

Now, we can draw the European transmission network:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
nx.draw(N, pos=pos, node_size=0)

You can already see that not all parts of the Network are connected with each other. Ireland, Great Britain, Scandinavia, and some Islands in the Mediterranean are not connected to the continental grid. At least not via AC transmission lines. They are through HVDC links and back-to-back converters. These subgraphs denote the different *synchronous zones* of the European transmission network:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d7/European_electricity_grid.svg/1920px-European_electricity_grid.svg.png" width="600px" />

:::{note}
The provided dataset is a bit outdated. Since its creation, the Baltic states have been synchronized with the continental European grid.
:::

In [ ]:
len(list(nx.connected_components(N)))

Let's build subgraphs for the synchronous zones by iterating over the connected components:

In [ ]:
subgraphs = []
for c in nx.connected_components(N):
    subgraphs.append(N.subgraph(c).copy())

We can now color-code them in the network plot:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
colors = ["red", "blue", "green", "orange", "teal", "cyan", "black"]
for i, sub in enumerate(subgraphs):
    sub_pos = {k: v for k, v in pos.items() if k in sub.nodes}
    nx.draw(sub, pos=sub_pos, node_size=0, edge_color=colors[i])

## Exercises

### Exercise 1

**Task 1:** Build the following graph in `networkx` using the functions to add graphs. The weight of each edge should correspond to the sum of the node labels it connects.

**Task 2:** Draw the finished graph with `matplotlib`.

<img src="https://github.com/fneum/data-science-for-esm/raw/main/data-science-for-esm/example-graph.png" width="500px" />

In [ ]:
G = nx.Graph()

In [ ]:
edges = [
    (0, 1, dict(weight=1)),
    (0, 2, dict(weight=2)),
    (0, 3, dict(weight=3)),
    (0, 4, dict(weight=4)),
    (1, 2, dict(weight=3)),
    (1, 4, dict(weight=5)),
    (2, 3, dict(weight=5)),
    (3, 4, dict(weight=7)),
]

In [ ]:
G.add_edges_from(edges)

In [ ]:
nx.draw(G, with_labels=True)

## Exercise 2

Reconsider the example of the European transmission network graph. Use the following code in a fresh notebook to get started:

```python
url = "https://tubcloud.tu-berlin.de/s/FmFrJkiWpg2QcQA/download/edges.csv"
edges = pd.read_csv(url, index_col=0)

G = nx.from_pandas_edgelist(edges, "bus0", "bus1", edge_attr=["x_pu", "s_nom"])

subgraphs = []
for c in nx.connected_components(G):
    subgraphs.append(G.subgraph(c).copy())
```

Choose one of the subgraphs (each representing a synchronous zone) and perform the following analyses:

In [ ]:
G = subgraphs[3]  # vary the subgraph here

**Task 1:** Determine the number of transmission lines and buses.

In [ ]:
len(G.edges)

In [ ]:
len(G.nodes)

**Task 2:** Check whether the network is planar.

In [ ]:
nx.is_planar(G)

**Task 3:** Calculate the average number of transmission lines connecting to a bus.

In [ ]:
pd.Series({k: v for k, v in G.degree}).mean()

**Task 4:** Determine the number of cycles forming the cycle basis.

In [ ]:
len(nx.cycle_basis(G))

**Task 5:** Create a histogram of the length of the cycles (i.e. number of *edges* per cycle) in the cycle basis.

In [ ]:
cycle_length = pd.Series([len(c) for c in nx.cycle_basis(G)])

In [ ]:
cycle_length.plot.hist(bins=100)

In [ ]:
# alternative
pd.Series([len(c) for c in nx.cycle_basis(G)]).value_counts().sort_index().plot()

**Task 6:** Calculate the average length of the cycles in the cycle basis.

In [ ]:
cycle_length.describe()

**Task 7:** Obtain the directed adjacency matrix.

In [ ]:
nx.adjacency_matrix(G).todense()

**Task 8:** Obtain the directed incidence matrix.

In [ ]:
K = nx.incidence_matrix(G).todense()
K